In [ ]:
import os
NEMO_ROOT = os.getcwd()
print(NEMO_ROOT)
import glob
import subprocess
import tarfile
import wget
import torch
import pytorch_lightning as pl

data_dir = os.path.join(NEMO_ROOT,'data')
os.makedirs(data_dir, exist_ok=True)


## Path to manifest files


In [ ]:
train_manifest = 'train_manifest.json'
validation_manifest = 'dev_manifest.json'
test_manifest = 'test_manifest.json'

# Training
Import necessary packages

Note: All the following steps are just for explanation of each section, but one can use the provided [training script](https://github.com/NVIDIA/NeMo/blob/main/examples/speaker_tasks/recognition/speaker_reco.py) to launch training in the command line.

In [ ]:
import nemo
# NeMo's ASR collection - This collection contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

## Model Configuration 
The TitaNet model is defined in a config file which declares multiple important sections.

They are:

1) model: All arguments that will relate to the Model - preprocessors, encoder, decoder, optimizer and schedulers, datasets, and any other related information

2) trainer: Any argument to be passed to PyTorch Lightning

In [ ]:
# This line will print the entire config of sample TitaNet model
!mkdir conf 
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/recognition/conf/titanet-large.yaml

In [ ]:
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/titanet-large.yaml')
config = OmegaConf.load(MODEL_CONFIG)
config.model.train_ds.manifest_filepath = train_manifest
config.model.validation_ds.manifest_filepath = validation_manifest


In [ ]:
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/SpeakerNet_recognition_3x2x512.yaml')
config = OmegaConf.load(MODEL_CONFIG)
config.model.train_ds.manifest_filepath = train_manifest
config.model.validation_ds.manifest_filepath = validation_manifest
config.model.test_ds.manifest_filepath = test_manifest

In [ ]:
print(OmegaConf.to_yaml(config))

Note: Since we are training speaker embedding extractor model for verification we do not add test_ds dataset. To include it add it to config and replace manifest file as 
`config.model.test_ds.manifest_filepath = test_manifest`

In [ ]:
config.model.decoder.num_classes = 2

## Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let us first instantiate a Trainer object!

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator

# Reduces maximum number of epochs to 5 for quick demonstration
config.trainer.max_epochs = 50

# Remove distributed training flags
config.trainer.strategy = None

# Remove augmentations
config.model.train_ds.augmentor=None

In [ ]:
trainer = pl.Trainer(**config.trainer)

## Setting up a NeMo Experiment
NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it !

In [ ]:
from nemo.utils.exp_manager import exp_manager
log_dir = exp_manager(trainer, config.get("exp_manager", None))
# The log_dir provides a path to the current logging directory for easy access
print(log_dir)

In [ ]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel(cfg=config.model, trainer=trainer)

In [ ]:
trainer.fit(speaker_model)

If you have a test manifest file, we can easily compute test accuracy by running
<pre><code>trainer.test(speaker_model, ckpt_path=None)
</code></pre>


In [ ]:
trainer.test(speaker_model, ckpt_path=None)

## Saving/Restoring a checkpoint
There are multiple ways to save and load models in NeMo. Since all NeMo models are inherently Lightning Modules, we can use the standard way that PyTorch Lightning saves and restores models.

NeMo also provides a more advanced model save/restore format, which encapsulates all the parts of the model that are required to restore that model for immediate use.

In this example, we will explore both ways of saving and restoring models, but we will focus on the PyTorch Lightning method.

## Saving and Restoring via PyTorch Lightning Checkpoints
When using NeMo for training, it is advisable to utilize the exp_manager framework. It is tasked with handling checkpointing and logging (Tensorboard as well as WandB optionally!), as well as dealing with multi-node and multi-GPU logging.

Since we utilized the exp_manager framework above, we have access to the directory where the checkpoints exist.

exp_manager with the default settings will save multiple checkpoints for us -

1) A few checkpoints from certain steps of training. They will have --val_loss= tags

2) Checkpoints at the last epoch of training are denoted by --last.

3) If the model finishes training, it will also have a --last checkpoint.

In [ ]:
# Let us list all the checkpoints we have
checkpoint_dir = os.path.join(log_dir, 'checkpoints')
checkpoint_paths = list(glob.glob(os.path.join(checkpoint_dir, "*.ckpt")))
checkpoint_paths

In [ ]:
final_checkpoint = list(filter(lambda x: "-last.ckpt" in x, checkpoint_paths))[0]
print(final_checkpoint)


## Restoring from a PyTorch Lightning checkpoint
To restore a model using the LightningModule.load_from_checkpoint() class method.

In [ ]:
restored_model = nemo_asr.models.EncDecSpeakerLabelModel.load_from_checkpoint(final_checkpoint)

# Finetuning
Since we don't have any new manifest file to finetune, I will demonstrate here by using the test manifest file we created earlier. 
an4 test dataset has a different set of speakers from the train set (total number: 10). And as we didn't split this dataset for validation I will use the same for validation. 

There are a couple of ways we can finetune a speaker recognition model. 
1. Finetuning using a pretrained model published on NGC. 
2. Finetuning from a PTL checkpoint. 

Since finetuning from a large pretrained model is more common, I shall use it to demonstrate finetuning procedure. In order to make finetuning step independent from training from scratch, we use another config. Here we shall use `titanet-finetune.yaml` config, that is created to show finetuning on pretrained titanet-large model. 

Note: You may use [finetune-script](https://github.com/NVIDIA/NeMo/blob/main/examples/speaker_tasks/recognition/speaker_reco_finetune.py) to launch training in the command line. Following is just a demonstration of the script

In [ ]:
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/recognition/conf/titanet-finetune.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/titanet-finetune.yaml')
finetune_config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(finetune_config))

For step 2, if one would like to finetune from a PTL checkpoint, `init_from_pretrained_model` in config should be replaced with `init_from_nemo_model` and need to provide the path to checkpoint. 

In [ ]:
finetune_config.model.train_ds.batch_size = 1

In [ ]:
finetune_config.model.train_ds.manifest_filepath = test_manifest
finetune_config.model.validation_ds.manifest_filepath = test_manifest
finetune_config.model.decoder.num_classes = 2

So we have set up the data and changed the decoder required for finetune, we now just need to create a trainer and start training with a smaller learning rate for fewer epochs

In [ ]:
# Setup the new trainer object
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'

trainer_config = OmegaConf.create(dict(
    devices=1,
    accelerator=accelerator,
    max_epochs=5,
    max_steps=-1,  # computed at runtime if not set
    num_nodes=1,
    accumulate_grad_batches=1,
    enable_checkpointing=False,  # Provided by exp_manager
    logger=False,  # Provided by exp_manager
    log_every_n_steps=1,  # Interval of logging.
    val_check_interval=1.0,  # Set to 0.25 to check 4 times per epoch, or an int for number of iterations
))
print(OmegaConf.to_yaml(trainer_config))

In [ ]:
trainer_finetune = pl.Trainer(**trainer_config)

## Setting the trainer to the restored model
Setting the trainer to the restored model

In [ ]:
from nemo.utils.exp_manager import exp_manager
log_dir_finetune = exp_manager(trainer_finetune, trainer_config.get("exp_manager", None))
print(log_dir_finetune)

## Fine-tune training step

When fine-tuning on a truly new dataset, we will not see such a dramatic improvement in performance. However, it should still converge a little faster than if it was trained from scratch.

In [ ]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel(cfg=finetune_config.model, trainer=trainer_finetune)
speaker_model.maybe_init_from_pretrained_checkpoint(finetune_config)

In the config, we keep weights of preprocessor and encoder, and attach a new decoder as mentioned in above section to match num of classes of new data

In [ ]:
## Fine-tuning for 5 epochs¶
trainer_finetune.fit(speaker_model)

Tip: Add more data augmentation and dropout while finetuning on your data

# Saving .nemo file
Now we can save the whole config and model parameters in a single .nemo and we can anytime restore from it

In [ ]:
restored_model.save_to(os.path.join(log_dir_finetune, '..',"titanet-large-finetune.nemo"))

In [ ]:
!ls {log_dir_finetune}/..

In [ ]:
# restore from a save model
restored_model = nemo_asr.models.EncDecSpeakerLabelModel.restore_from(os.path.join(log_dir_finetune, '..', "titanet-large-finetune.nemo"))


# Speaker Verification

Training for a speaker verification model is almost the same as the speaker recognition model with a change in the loss function. Angular Loss is a better function to train for a speaker verification model as the model is trained in an end-to-end manner with loss optimizing for embeddings cluster to be far from each other for different speaker by maximizing the angle between these clusters

To train for verification we just need to toggle `angular` flag in `config.model.decoder.params.angular = True` else set it to `False` to train with cross-entropy loss for identification purposes. 
Once we set this, the loss will be changed to angular loss and we can follow the above steps to the model.
Note the scale and margin values to be set for the loss function are present at `config.model.loss.scale` and `config.model.loss.margin`

## Extract Speaker Embeddings
Once you have a trained model or use one of our pretrained nemo checkpoints to get speaker embeddings for any speaker.

To demonstrate this we shall use `nemo_asr.models.EncDecSpeakerLabelModel` with say 5 audio_samples from our dev manifest set. This model is specifically for inference purposes to extract embeddings from a trained `.nemo` model

In [ ]:
verification_model = nemo_asr.models.EncDecSpeakerLabelModel.restore_from(os.path.join(log_dir_finetune, '..', 'titanet-large-finetune.nemo'))

Now, we need to pass the necessary manifest_filepath and params to set up the data loader for extracting embeddings

In [ ]:
!head -5 {validation_manifest} > embeddings_manifest.json

In [ ]:
config.model.train_ds

In [ ]:
from nemo.collections.asr.parts.utils.speaker_utils import embedding_normalize
from tqdm import  tqdm
try:
    from torch.cuda.amp import autocast
except ImportError:
    from contextlib import contextmanager

    @contextmanager
    def autocast(enabled=None):
        yield
import numpy as np
import json
import pickle as pkl

In [ ]:
def get_embeddings(speaker_model, manifest_file, batch_size=1, embedding_dir='./', device='cuda'):
    test_config = OmegaConf.create(
        dict(
            manifest_filepath=manifest_file,
            sample_rate=16000,
            labels=None,
            batch_size=batch_size,
            shuffle=False,
            time_length=20,
        )
    )

    speaker_model.setup_test_data(test_config)
    speaker_model = speaker_model.to(device)
    speaker_model.eval()

    all_embs=[]
    out_embeddings = {}
           
    for test_batch in tqdm(speaker_model.test_dataloader()):
        test_batch = [x.to(device) for x in test_batch]
        audio_signal, audio_signal_len, labels, slices = test_batch
        with autocast():
            _, embs = speaker_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
            emb_shape = embs.shape[-1]
            embs = embs.view(-1, emb_shape)
            all_embs.extend(embs.cpu().detach().numpy())
        del test_batch

    all_embs = np.asarray(all_embs)
    all_embs = embedding_normalize(all_embs)
    with open(manifest_file, 'r') as manifest:
        for i, line in enumerate(manifest.readlines()):
            line = line.strip()
            dic = json.loads(line)
            uniq_name = '@'.join(dic['audio_filepath'].split('/')[-3:])
            out_embeddings[uniq_name] = all_embs[i]

    embedding_dir = os.path.join(embedding_dir, 'embeddings')
    if not os.path.exists(embedding_dir):
        os.makedirs(embedding_dir, exist_ok=True)

    prefix = manifest_file.split('/')[-1].rsplit('.', 1)[-2]

    name = os.path.join(embedding_dir, prefix)
    embeddings_file = name + '_embeddings.pkl'
    pkl.dump(out_embeddings, open(embeddings_file, 'wb'))
    print("Saved embedding files to {}".format(embedding_dir))

In [ ]:
manifest_filepath = os.path.join(NEMO_ROOT,'embeddings_manifest.json')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
get_embeddings(verification_model, manifest_filepath, batch_size=64,embedding_dir='./', device=device)

Embeddings are stored in dict structure with key-value pair, key being uniq_name generated based on audio_filepath of the sample present in manifest_file in `embedding_dir`

In [ ]:
ls ./embeddings/

In [ ]:
from nemo.collections.asr.models.label_models import EncDecSpeakerLabelModel
embeddings_model = EncDecSpeakerLabelModel.from_pretrained("titanet_large")
#embs = embeddings_model.get_embedding("segments_dir/speaker_0_0.0_4200.0.wav")   

In [62]:
import json
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load the manifests
with open('train_manifest.json', 'r') as f:
    train_manifest = [json.loads(l) for l in f.readlines()] 
with open('dev_manifest.json', 'r') as f:
    val_manifest = [json.loads(l) for l in f.readlines()] 



In [63]:
# Prepare your data
features = []
labels = []

for item in train_manifest:
    # # Load audio file
    embeddings = embeddings_model.get_embedding(item['audio_filepath'])

    # # Extract MFCC features
    # mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # # Use the mean and standard deviation of the MFCCs as features
    # mfccs_mean = np.mean(mfccs.T, axis=0)
    # mfccs_std = np.std(mfccs.T, axis=0)
    features.append(embeddings.squeeze().cpu().numpy())
    labels.append(item['label'])

# Convert labels to integers
le = LabelEncoder()
labels = le.fit_transform(labels)


In [64]:
# Split data into training and test sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2)

In [65]:

# Train a simple MLP classifier
# model = MLPClassifier(hidden_layer_sizes=(100, 100, ), max_iter=1000, verbose=True)
model = SGDClassifier(loss="log_loss", verbose=True)
model.fit(features_train, labels_train)

-- Epoch 1
Norm: 28.50, NNZs: 192, Bias: 0.635026, T: 348, Avg. loss: 0.194852
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 28.65, NNZs: 192, Bias: -0.026748, T: 696, Avg. loss: 0.026112
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 28.60, NNZs: 192, Bias: 0.131816, T: 1044, Avg. loss: 0.025887
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 28.44, NNZs: 192, Bias: 0.541714, T: 1392, Avg. loss: 0.024670
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 28.30, NNZs: 192, Bias: 0.575395, T: 1740, Avg. loss: 0.024681
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 28.20, NNZs: 192, Bias: 0.034714, T: 2088, Avg. loss: 0.024686
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 28.12, NNZs: 192, Bias: 0.147501, T: 2436, Avg. loss: 0.023813
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 28.07, NNZs: 192, Bias: 0.881832, T: 2784, Avg. loss: 0.024828
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 28.02, NNZs: 192, Bias: 0.876313, T: 3132, Avg. loss: 0.023994
T

SGDClassifier(loss='log_loss', verbose=True)

In [66]:

# Prepare validation data and evaluate the model
features_val = []
labels_val = []

for item in val_manifest:
    # Load audio file
    embeddings = embeddings_model.get_embedding(item['audio_filepath'])
    features_val.append(embeddings.squeeze().cpu().numpy())
    labels_val.append(item['label'])

labels_val = le.transform(labels_val)

# Predict labels for validation set and calculate accuracy
labels_pred = model.predict(features_val)
print("Accuracy:", accuracy_score(labels_val, labels_pred))

Accuracy: 1.0


In [59]:
import pandas as pd

In [67]:
from IPython.display import Audio, display

# Assuming 'X_test' is your test data and 'y_test' are the true labels
# and 'model' is your trained model

labels_pred = model.predict(features_val)
X_test = pd.DataFrame.from_records(val_manifest)
y_pred = labels_pred
y_test = labels_val

incorrect_samples = X_test[y_pred != y_test]
incorrect_true_labels = y_test[y_pred != y_test]
incorrect_pred_labels = y_pred[y_pred != y_test]

# To play some of these incorrect samples:
for i in range(min(10, len(incorrect_samples))):  # play 10 or less
    print(f'True Label: {incorrect_true_labels[i]}')
    print(f'Predicted Label: {incorrect_pred_labels[i]}')
    print(f'Audio: {incorrect_samples.iloc[i].audio_filepath}')
    display(Audio(incorrect_samples.iloc[i].audio_filepath))


In [68]:
import pickle

In [69]:
with open("grey_vs_brady.pkl", "wb") as f:
    pickle.dump(model, f)